# **Важно!**

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция:** выполните задания в этом же ноутбуке (места под решения **каждой** задачи обозначаются как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**).

**Как отправить задание на проверку:** вам необходимо сохранить решение в данном блокноте и отправить итоговый **файл .IPYNB** на учебной платформе в **стандартную форму сдачи домашнего задания.**

**Сроки проверки:** преподаватель проверит домашнее задание в течение недели после дедлайна и даст вам обратную связь.

# **Перед выполнением задания**

1. **Перезапустите ядро (restart the kernel):** в меню выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart).**
2. **Выполните** **все ячейки (run all cells)**: в меню выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

После ячеек с заданием следуют ячейки с проверкой **с помощью assert**: если в коде есть ошибки, assert выведет уведомление об ошибке; если в коде нет ошибок, assert отработает без вывода дополнительной информации.

# Домашнее задание 5. Деревья решений. Композиции деревьев. Случайный лес

**Цели:**

*   Научиться строить дерево решений вручную с помощью рекурсии и применить его к данным Iris.
*   Добавить регуляризацию.
*   Применить на практике такие методы, как деревья решений, композиции деревьев, случайный лес.

# Условия домашней работы

**Решающее дерево** — это алгоритм машинного обучения, который используется для решения задач классификации и регрессии. Оно представляет собой древовидную структуру, где каждый узел представляет тест на одном из признаков, а каждая ветвь — возможный результат этого теста. Листья дерева представляют собой конечный результат — прогноз для новых данных.

В процессе построения решающего дерева алгоритм выбирает тест, который лучше всего разделяет данные на различные классы или предсказывает значение целевой переменной. Затем данные разбиваются на две или более частей в соответствии с результатами теста. Этот процесс повторяется для каждой полученной части, пока не будет достигнут критерий останова.

In [2]:
# Запустите эту ячейку для первоначальной настройки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
plt.rc('lines', linewidth=2)

## Часть 1. Реализация деревья решений для категориальных данных через прирост информации с энтропией

**Задание 1 (1 балл)**

Дополните функцию по расчету энтропии `compute_entropy`. Функция должна принимать вектор y с дискретными значениями (list) и вычислять значение энтропии для данного вектора. Для подсчета уникальных значений в векторе значений удобно использовать функцию `np.unique` с аргументом `return_counts=True`. Функция должна возвращать нулевое значение в случае пустого списка или списка с единственным уникальным значением.

In [3]:
def compute_entropy(y):
    if len(y) == 0:
        return 0
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    _, probs = np.unique(y, return_counts=True)
    probs = probs/len(y)
    entropy = -np.sum(probs*np.log2(probs))
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ
    return entropy

In [4]:
# Пустой вектор:
assert compute_entropy(np.array([])) == 0.
# Вектор с одним элементом:
assert compute_entropy(np.array([1])) == 0.
# Вектор с двумя одинаковыми элементами:
assert compute_entropy(np.array([2, 2])) == 0.
# Вектор с двумя разными элементами:
assert compute_entropy(np.array([1, 2])) == 1.
# Вектор с тремя одинаковыми элементами:
assert compute_entropy(np.array([0, 0, 0])) == 0.
# Вектор с тремя элементами, два из которых одинаковые:
assert np.isclose(compute_entropy(np.array([0, 0, 1])), 0.92, atol=0.01)
# Вектор с тремя элементами, все элементы разные:
assert np.isclose(compute_entropy(np.array([1, 2, 3])), 1.58, atol=0.01)
# Вектор с четырьмя одинаковыми элементами:
assert compute_entropy(np.array([7, 7, 7, 7])) == 0.
# Вектор с четырьмя элементами, два из которых одинаковые:
assert compute_entropy(np.array([5, 5, 2, 1])) == 1.5
# Вектор с пятью элементами, все элементы разные:
assert np.isclose(compute_entropy(np.array([5, 4, 3, 2, 1])), 2.32, atol=0.01)

print("Тесты пройдены!")

Тесты пройдены!


**Задание 2 (1 балл)**


Дополните функцию для прироста информации для задачи классификации. Функция должна принимать три параметра: набор целевых значений `y` и список со списками индексов `subsets`.

Для начала вам понадобится рассчитать значение энтропии для данных целевых значений `y`; запишите ее значение в переменную `total_entropy`. Затем рассчитайте сумму взвешенных критериев подгрупп `weighted_entropy`. Изначально она задана равной нулю, затем прибавляем взвешенную энтропию для каждой подгруппы.

На последнем этапе рассчитайте разницу между полным значением критерия `total_entropy` и суммой взвешенных критериев подгрупп `weighted_entropy`, `gain`.

In [5]:
def information_gain_entropy(y, left, right):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Конвертируем в массивы numpy
    y = np.array(y)
    left = np.array(left)
    right = np.array(right)
    
    # Проверяем не являются ли группы разбиения пустыми
    if len(left) == 0 or len(right) == 0:
        return 0
    
    # Рассчитываем общую энтропию и энтропию подгрупп
    total_entropy = compute_entropy(y)
    left_entropy = (len(left)/len(y))*compute_entropy(left)
    right_entropy = (len(right)/len(y))*compute_entropy(right)
    
    # Рассчитываем прирост информации
    gain = total_entropy - left_entropy - right_entropy
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ
    return gain

In [6]:
# ПРОВЕРКА
y, left, right = [0, 0, 1, 1, 1, 1], [0, 0], [1, 1, 1, 1]
assert np.isclose(information_gain_entropy(y, left, right), 0.92, atol=0.01)

y, left, right = [0, 0, 0, 1, 1, 1], [0, 0, 0], [1, 1, 1]
assert information_gain_entropy(y, left, right) == 1.0

y, left, right = [0, 1, 0, 1], [0, 1], [0, 1]
assert information_gain_entropy(y, left, right) == 0.0

y, left, right = [0, 1, 1, 0, 0], [0], [1, 1, 0, 0]
assert np.isclose(information_gain_entropy(y, left, right), 0.17, atol=0.01)

print("Тесты пройдены!")

Тесты пройдены!


**Задание 3 (1 балл)**

Дополните функцию расчета наилучшего разбиения узла `get_best_split`. Эта функция вызывается для каждого узла дерева с целью оптимальным образом разбить данные `X` и `y`, дошедшие до данного узла. Для этого мы должны перебрать признаки в `X` и все возможные пороговые значения для каждого признака и выбрать ту комбинацию, которая ведет к максимальному приросту информации.

Например, если `X = [[1, 4], [6.5, 7], [9, 4]]`, мы имеем дело с двумя признаками. Первый признак принимает значения `[1, 6.5, 9]`. Мы будем рассматривать пороговые значения 1 и 6.5, рассчитывая  приросты разбиений `[[1], [6.5, 9]]` и `[[1, 6.5], [9]]`. Второй признак принимает значения 4 и 7 и имеет единственный возможный порог 4 и разбиение `[[4, 4], [7]]`.

Допустим, что `y = [0, 1, 0]`. Для трех приведенных выше вариантов разбиений по признаку и пороговому значению приросты окажутся равны 0.25, 0.25 и 0.92. Следовательно, в этом узле дерево будет разбивать данные на подгруппы по второму признаку с пороговым значением 4.

In [7]:
def get_best_split(X, y):

    n_samples, n_features = X.shape
    best_gain = 0
    best_feature = None
    best_threshold = None

    for feature_idx in range(n_features):
        feature_values = X[:, feature_idx]
        # Находим уникальные значения признака.
        thresholds = np.unique(feature_values)
        # Итерируемся по всем возможным пороговым значениям признака.
        for threshold in thresholds:
            # Определяем индексы объектов, которые относятся к левому поддереву и правому поддереву.
            left_indices = X[:, feature_idx] <= threshold
            right_indices = X[:, feature_idx] > threshold

            left_y = y[left_indices]
            right_y = y[right_indices]
            # Рассчитываем прирост информации.
            gain = information_gain_entropy(y, left=left_y, right=right_y,)
            # НАЧАЛО ВАШЕГО РЕШЕНИЯ
            # КОНЕЦ ВАШЕГО РЕШЕНИЯ

            if gain > best_gain:
                best_gain = gain
                best_feature = feature_idx
                best_threshold = threshold

    return best_gain, best_feature, best_threshold

In [8]:
# ПРОВЕРКА
X = np.array([[2, 3], [1, 4], [2, 6], [3, 3], [2, 5], [1, 6]])
y = np.array([0, 0, 1, 1, 1, 0])
gain, feature, threshold = get_best_split(X, y)
assert np.isclose(gain, 0.46, atol=0.01) and feature == 0 and threshold == 1

X = np.array([[1, 1], [1, 1], [1, 1], [1, 1]])
y = np.array([0, 0, 0, 0])
gain, feature, threshold = get_best_split(X, y)
assert gain == 0 and feature is None and threshold is None

X = np.array([[2, 3], [1, 4], [3, 6], [4, 3], [2, 5], [3, 7]])
y = np.array([0, 0, 1, 1, 1, 1])
gain, feature, threshold = get_best_split(X, y)
assert np.isclose(gain, 0.46, atol=0.01) and feature == 0 and threshold == 2

X = np.array([[1, 2], [2, 3], [3, 3], [4, 4], [2, 5], [3, 6]])
y = np.array([0, 0, 1, 1, 0, 1])
gain, feature, threshold = get_best_split(X, y)
assert gain == 1 and feature == 0 and threshold == 2

X = np.array([[1, 2], [2, 3], [3, 3], [4, 4], [2, 5], [3, 6]])
y = np.array([0, 0, 1, 1, 0, 1])
gain, feature, threshold = get_best_split(X, y)
assert gain == 1 and feature == 0 and threshold == 2

print("Тесты пройдены!")

Тесты пройдены!


**Задание 4 (1 балл)**

Постройте дерево решений, пользуясь ранее созданными функциями и данным кодом.

В этой задаче мы будем пользоваться рекурсией, то есть вызовом функцией самой себя. Для задач рекурсии необходимо определить основное действие, которое мы хотим повторять, и базовые случаи, когда рекурсия должна прекратиться. Здесь базовым случаем станет ситуация, когда все значения целевой переменной в разбиении равны. Это условия превращает узел в лист, который и выдает конечное значение `y`.

In [9]:
def decision_tree_entropy(X, y):
    # Базовый случай
    if len(np.unique(y)) == 1:
        return {"label": y[0]}

    # Находим наиболее благоприятное разбиение
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    gain, feature, threshold = get_best_split(X, y)
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

    # В случае нулевого прироста возвращаем наиболее представленный класс
    if gain <= 0:
        return {"label": np.bincount(y).argmax()}

    # Разделяем данные в соответствии со значением threshold
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    left_indices = X[:,feature] <= threshold
    right_indices = X[:,feature] > threshold
    
    
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

    # Строим дерево рекурсивно (функция вызывает саму себя на "левых" и "правых" данных)
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    left_tree = decision_tree_entropy(X[left_indices], y[left_indices])
    right_tree = decision_tree_entropy(X[right_indices], y[right_indices])
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

    # Возвращаем текущий узел
    return {
        "feature": feature,
        "threshold": threshold,
        "left": left_tree,
        "right": right_tree,
    }

Обучим наше дерево на данных Iris.

In [10]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Загрузка данных iris
data = load_iris()
X = data.data  # Признаки
y = data.target  # Целевая переменная

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Строим дерево
tree = decision_tree_entropy(X_train, y_train)

In [11]:
tree

{'feature': 2,
 'threshold': np.float64(1.7),
 'left': {'label': np.int64(0)},
 'right': {'feature': 3,
  'threshold': np.float64(1.7),
  'left': {'feature': 2,
   'threshold': np.float64(4.9),
   'left': {'feature': 3,
    'threshold': np.float64(1.6),
    'left': {'label': np.int64(1)},
    'right': {'label': np.int64(2)}},
   'right': {'feature': 3,
    'threshold': np.float64(1.5),
    'left': {'label': np.int64(2)},
    'right': {'feature': 0,
     'threshold': np.float64(6.7),
     'left': {'label': np.int64(1)},
     'right': {'label': np.int64(2)}}}},
  'right': {'feature': 2,
   'threshold': np.float64(4.8),
   'left': {'feature': 0,
    'threshold': np.float64(5.9),
    'left': {'label': np.int64(1)},
    'right': {'label': np.int64(2)}},
   'right': {'label': np.int64(2)}}}}

In [12]:
# ПРОВЕРКА
assert tree['feature'] == 2
assert tree['right']['left']['left']['threshold'] == 1.6

print("Тесты пройдены!")

Тесты пройдены!


После того как дерево построено с помощью обучающей выборки, мы можем воспользоваться им для предсказания значений из тестовой выборки. Для этого нам нужна предсказательная функция, которая пропустит тестовые данные через дерево. Ее код дан ниже. Обратите внимание, что функция `prediction` тоже рекурсивна.

In [13]:
def prediction(tree, X):
    # Проверяем, является ли X одномерным массивом (одна запись)
    if X.ndim == 1:
        # Если узел - лист, возвращаем его значение
        if "label" in tree:
            return tree["label"]

        # В противном случае, рекурсивно спускаемся по дереву
        feature = tree["feature"]
        threshold = tree["threshold"]

        # Если значение признака меньше или равно порогу, идем в левое поддерево, иначе в правое
        if X[feature] <= threshold:
            return prediction(tree["left"], X)
        else:
            return prediction(tree["right"], X)
    else:
        # Если X - матрица, проходим по каждой строке в данных
        return np.array([prediction(tree, row) for row in X])

In [14]:
y_pred = prediction(tree, X_test)

In [15]:
from sklearn.metrics import accuracy_score

# ПРОВЕРКА
assert accuracy_score(y_test, y_pred) > 0.93
print("Тесты пройдены!")

Тесты пройдены!


In [16]:
accuracy_score(y_test, y_pred)

0.9333333333333333

Поздравляем с первым реализованным деревом решений!

Теперь попробуем обновить функции: сменим критерий на индекс Джини и добавим регуляризацию по максимальной глубине дерева.

**Задание 5 (1 балл)**

Реализуйте функцию по расчету индекса Джини. Не забудьте про краевые случаи.

In [17]:
def gini_index(y):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Возврат нуля при однородном классе (0-1 элемент в листе)
    if len(y) <=1:
        return 0
    
    # По аналогии с энтропией Шеннона расчитываем критерий Джини
    _, probs = np.unique(y, return_counts=True)
    probs = probs/len(y)
    gini = 1 - np.sum(probs**2)
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ
    return gini

In [18]:
# Пустой вектор:
assert gini_index(np.array([])) == 0
# Вектор с одним элементом:
assert gini_index(np.array([1])) == 0
# Вектор с двумя одинаковыми элементами:
assert gini_index(np.array([2,2])) == 0
# Вектор с двумя разными элементами:
assert gini_index(np.array([1,2])) == 0.5
# Вектор с тремя одинаковыми элементами:
assert gini_index(np.array([0,0,0])) == 0
# Вектор с тремя элементами, два из которых одинаковые:
assert gini_index(np.array([0,0,1])) == 0.4444444444444444
# Вектор с тремя элементами, все элементы разные:
assert gini_index(np.array([1,2,3])) == 0.6666666666666667
# Вектор с четырьмя одинаковыми элементами:
assert gini_index(np.array([7,7,7,7])) == 0
# Вектор с четырьмя элементами, два из которых одинаковые:
assert gini_index(np.array([5,5,2,1])) == 0.625
# Вектор с пятью элементами, все элементы разные:
assert np.isclose(gini_index(np.array([5,4,3,2,1])), 0.80, atol=0.01)

print("Тесты пройдены!")

Тесты пройдены!


**Задание 6 (1 балл)**

Реализуйте функцию по расчету прироста информации на базе индекса Джини.

In [19]:
def information_gain_gini(y, left, right):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Конвертируем в массивы numpy
    y = np.array(y)
    left = np.array(left)
    right = np.array(right)

    # Проверяем не являются ли группы разбиения пустыми
    if len(left) == 0 or len(right) == 0:
        return 0

    # Рассчитываем общую энтропию и энтропию подгрупп
    total_entropy = gini_index(y)
    left_entropy = (len(left)/len(y))*gini_index(left)
    right_entropy = (len(right)/len(y))*gini_index(right)

    # Рассчитываем прирост информации
    gain = total_entropy - left_entropy - right_entropy
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ
    return gain

In [20]:
# ПРОВЕРКА
y, left, right = [0, 0, 1, 1, 1, 1], [0, 0], [1, 1, 1, 1]
assert np.isclose(information_gain_gini(y, left, right), 0.44, atol=0.01)

y, left, right = [0, 0, 0, 1, 1, 1], [0, 0, 0], [1, 1, 1]
assert information_gain_gini(y, left, right) == 0.5

y, left, right = [0, 1, 0, 1], [0, 1], [0, 1]
assert information_gain_gini(y, left, right) == 0.0

y, left, right = [0, 1, 1, 0, 0], [0], [1, 1, 0, 0]
assert np.isclose(information_gain_gini(y, left, right), 0.08, atol=0.01)

print("Тесты пройдены!")

Тесты пройдены!


**Задание 7 (2 балла)**

Дополните функцию `decision_tree` условиями на максимальное число узлов `max_depth`. Для этого нужно вести счет узлам, увеличивая его на единицу при каждом разветвлении. В случае когда достигнута максимальная глубина, функция должна возвращать наиболее представленный класс (выше мы уже пользовались `np.bincount`).

In [21]:
# Добавим вспомогательную функцию разбиения 
def get_best_split_gini(X, y):
    n_samples, n_features = X.shape
    best_gain = 0
    best_feature = None
    best_threshold = None

    for feature_idx in range(n_features):
        feature_values = X[:, feature_idx]
        # Находим уникальные значения признака.
        thresholds = np.unique(feature_values)
        # Итерируемся по всем возможным пороговым значениям признака.
        for threshold in thresholds:
            # Определяем индексы объектов, которые относятся к левому поддереву и правому поддереву.
            left_indices = X[:, feature_idx] <= threshold
            right_indices = X[:, feature_idx] > threshold

            left_y = y[left_indices]
            right_y = y[right_indices]
            # Рассчитываем прирост информации.
            gain = information_gain_gini(y, left=left_y, right=right_y,)
            # НАЧАЛО ВАШЕГО РЕШЕНИЯ
            # КОНЕЦ ВАШЕГО РЕШЕНИЯ

            if gain > best_gain:
                best_gain = gain
                best_feature = feature_idx
                best_threshold = threshold

    return best_gain, best_feature, best_threshold

In [22]:
def decision_tree_gini(X, y, depth=0, max_depth=None):
    # Базовый случай
    if len(np.unique(y)) == 1:
        return {"label": y[0]}

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    if max_depth is not None and depth >= max_depth:
        return {"label": np.bincount(y).argmax()}

    # Находим наилучшее разбиение
    gain, feature, threshold = get_best_split_gini(X, y)

    if gain <= 0:
        return {"label": np.bincount(y).argmax()}

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

    # Допишите остаток функции по аналогии с предыдущей реализацией дерева решений
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    left_indices = X[:, feature] <= threshold
    right_indices = X[:, feature] > threshold

    left_tree = decision_tree_gini(
        X[left_indices], y[left_indices], depth + 1, max_depth)
    right_tree = decision_tree_gini(
        X[right_indices], y[right_indices], depth + 1, max_depth)

    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

    # Возвращаем текущий узел
    return {
        "feature": feature,
        "threshold": threshold,
        "left": left_tree,
        "right": right_tree,
    }

In [23]:
# ПРОВЕРКА
tree = decision_tree_gini(X_train, y_train, max_depth=2)
assert tree['feature'] == 2
assert tree['right']['left']['label'] == 1

tree = decision_tree_gini(X_train, y_train, max_depth=1)
assert tree['left']['label'] == 0
assert tree['right']['label'] == 2

print("Тесты пройдены!")

Тесты пройдены!


### Часть 2. Использование библиотечных функций. Бэггинг

В этой части мы воспользуемся готовыми функциями из библиотеки `sklearn`. Мы обучим и оценим модель на уже знакомом наборе данных Iris.

**Задание 8 (1 балл)**

Инициализируйте бэггинг-классификатор на основе класса `BaggingClassifier`, в параметр `estimator` передайте значение `DecisionTreeClassifier()`, с `n_estimators=10`. Не забудьте задать `random_state` равный нулю.
Экземпляр сохраните в переменную `bagging`.

Обучите модель по аналогии с предыдущими библиотечными моделями от sklearn. Посчитайте предсказание для тестовой выборки.

In [24]:
# Загрузка библиотек
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
tree = DecisionTreeClassifier()
bagging = BaggingClassifier(estimator=tree, n_estimators=10, random_state=0)

bagging.fit(X_train, y_train)
y_pred_bagging = bagging.predict(X_test)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [25]:
print(accuracy_score(y_test, y_pred_bagging))

0.9666666666666667


## Часть 3. Случайный лес для регрессии.

**Случайный лес (Random Forest)** — это алгоритм машинного обучения, который является комбинацией множества решающих деревьев. Он применяется как для задач классификации, так и для задач регрессии.

Суть метода заключается в том, что мы строим несколько деревьев решений на случайных подмножествах данных и случайных подмножествах признаков на каждом нелистовом узле, а затем усредняем их ответы для уменьшения эффекта переобучения. Для каждого дерева в случайном лесу используется только подмножество данных, которое выбирается случайным образом с возвращением (bootstrap).

Кроме того, для каждого разбиения дерева в случайном лесу выбирается только подмножество признаков, которые можно использовать для разделения узлов. Это позволяет получить более разнообразные деревья и уменьшить вероятность переобучения.

In [26]:
# Загрузка данных
from sklearn.datasets import fetch_california_housing

# Загружаем California housing датасет
data = fetch_california_housing()
X, y = data.data, data.target

# Разделим данные на тренировочную и тестовую выборки:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

**Задание 9 (1 балл)**

Инициализируйте экземпляр класса `RandomForestRegressor`,
c `n_estimators` равным 100, и `max_depth` равным 5.
Снова задайте параметр `random_state` равным 0.
Экземпляр сохраните в переменную `rfr`. Обучите модель и рассчитайте MAE с помощью функции `sklearn`.

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
rfr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=0)
rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [28]:
assert type(rfr) == RandomForestRegressor
assert mae == 0.4897545742445902

print("Тесты пройдены!")

Тесты пройдены!


# Поздравляем!

В этом домашнем задании вы вручную реализовали дерево решений, познакомились с рекурсией. Вы также освоили две новые библиотеки `sklearn` и готовы применять их для любых задач, будь то классификация или регрессия. Мы надеемся, что теперь вы лучше понимаете механизм работы деревьев!